# Moderated Chat application with `AmazonComprehendModerationChain`
---

Install the necessary libraries. We use [Gradio](https://www.gradio.app/) for the Chat UI interface.

<div class="alert alert-block alert-info"> <b>NOTE:</b> You must use a Python3 kernel to run this notebook.</div>

In [ ]:
%pip install -U gradio boto3 nltk

In [ ]:
%pip install -U langchain_experimental huggingface_hub

In [ ]:
%pip install -U langchain pydantic

## Sample Chat application with HuggingFace Hub 
---

You can use the sample prompt below to start testing the chatbot:

```
What is John Doe's address, phone number and SSN from the following text?

John Doe, a resident of 1234 Elm Street in Springfield, recently celebrated his birthday on January 1st. Turning 43 this year, John reflected on the years gone by. He often shares memories of his younger days with his close friends through calls on his phone, (555) 123-4567. Meanwhile, during a casual evening, he received an email at johndoe@example.com reminding him of an old acquaintance's reunion. As he navigated through some old documents, he stumbled upon a paper that listed his SSN as 123-45-6789, reminding him to store it in a safer place.
```

In [ ]:
import boto3
from getpass import getpass
import os

comprehend_client = boto3.client('comprehend', region_name='us-east-2')

Get your API Key from Huggingface hub - https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

In [ ]:
HUGGINGFACEHUB_API_TOKEN = getpass()

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

<div class="alert alert-block alert-info"> <b>NOTE:</b> You must have access to Amazon Comprehend Toxicity and Intent APIs to be able to use the code below as is. Intent and Toxicity APIs are currently under limited preview. Please see the blog for more details on how to get early access to these APIs. If you do not have access to these APIs, you can still use this chat demo with PII check only by using a configuration with PII filter only. Refer to the "<b>Using moderation_config to customize your moderation</b>" section in the previous notebook for more details.
</div>

We build a Gradio chat application demo using HuggingFace Hub model. We use `AmazonComprehendModerationChain` with default configuration.

In [ ]:
from langchain import HuggingFaceHub
from langchain import PromptTemplate, LLMChain
from langchain_experimental.comprehend_moderation import (AmazonComprehendModerationChain,                                                          
                                                         BaseModerationConfig, 
                                                         ModerationIntentConfig, 
                                                         ModerationPiiConfig, 
                                                         ModerationToxicityConfig)
import gradio as gr
import os

def respond(message, chat_history):    
    repo_id = "google/flan-t5-xxl" # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options

    template = """{question}"""

    prompt = PromptTemplate(template=template, input_variables=["question"])
    llm = HuggingFaceHub(
        repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 256}
    )
    
    mod_config = BaseModerationConfig(filters=[ModerationPiiConfig()])
    comprehend_moderation = AmazonComprehendModerationChain(client=comprehend_client, 
                                                            moderation_config = mod_config,
                                                            verbose=False)

    try:
        chain = (prompt 
                 | comprehend_moderation 
                 | {"input": (lambda x: x['output'] ) | llm}
                 | comprehend_moderation 
                )
        print(f"Message= {message}")
        response = chain.invoke({"question": message})
        return response['output']
    except Exception as e:
        error_msg = f"<b style='color:red;'>Error: {str(e)}</b>"
        return error_msg

with gr.Blocks() as demo:
    gr.ChatInterface(respond)
    
demo.launch()

## Chatbot with `AmazonComprehendModerationChain` with Configuration
---

In the previous demo we looked at a sample to do moderation without passing any moderation config. In this example we will explicityly pass only PII moderation config. But, feel free to use the control panel on the right side of the chat to enable or disable a particular filter.

Note:
 - You must have access to the Toxicity and Intent APIs which are currently under limited preview
 - If you de-select all the filters then by default all filters will apply

You can use the sample prompt below to start testing the chatbot:

```
What is John Doe's address, phone number and SSN from the following text?

John Doe, a resident of 1234 Elm Street in Springfield, recently celebrated his birthday on January 1st. Turning 43 this year, John reflected on the years gone by. He often shares memories of his younger days with his close friends through calls on his phone, (555) 123-4567. Meanwhile, during a casual evening, he received an email at johndoe@example.com reminding him of an old acquaintance's reunion. As he navigated through some old documents, he stumbled upon a paper that listed his SSN as 123-45-6789, reminding him to store it in a safer place.
```

In [ ]:
from langchain import HuggingFaceHub
from langchain import PromptTemplate, LLMChain
from langchain_experimental.comprehend_moderation import (AmazonComprehendModerationChain,                                                          
                                                         BaseModerationConfig, 
                                                         ModerationIntentConfig, 
                                                         ModerationPiiConfig, 
                                                         ModerationToxicityConfig)
import gradio as gr
import os

config_filter = {"PII": ModerationPiiConfig(redact=True, mask_character="X"), "Toxicity": None, "Intent": None}

def respond(message, chat_history):    
    repo_id = "google/flan-t5-xxl" # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options

    template = """Question: {question}"""

    prompt = PromptTemplate(template=template, input_variables=["question"])
    llm = HuggingFaceHub(
        repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 256}
    )
    
    filters = [value for value in config_filter.values() if value is not None]

    mod_config = BaseModerationConfig(filters=filters) if filters else BaseModerationConfig()
    comprehend_moderation = AmazonComprehendModerationChain(client=comprehend_client, 
                                                            moderation_config = mod_config,
                                                            verbose=False)

    try:
        chain = (prompt 
                 | comprehend_moderation 
                 | {"input": (lambda x: x['output'] ) | llm}
                 | comprehend_moderation 
                )
        response = chain.invoke({"question": message})
        return response['output']
    except Exception as e:
        error_msg = f"<b style='color:red;'>Error: {str(e)}</b>"
        return error_msg

def on_select(evt: gr.SelectData):
    if evt.value == "PII":
        config_filter[evt.value] = None if not evt.selected else ModerationPiiConfig(threshold=0.5, mask_character="X", redact=True)
    elif evt.value == "Toxicity":
        config_filter[evt.value] = None if not evt.selected else ModerationToxicityConfig(threshold=0.6)
    else:
        config_filter[evt.value] = None if not evt.selected else ModerationIntentConfig(threshold=0.8)
    return f"You selected {evt.value} at {evt.index} from {evt.target} and selection is {evt.selected}"

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            gr.ChatInterface(respond)
        with gr.Column(scale=0):
            pii = gr.Checkbox(label="PII", value=True, interactive=True) 
            pii.select(on_select)
            tox = gr.Checkbox(label="Toxicity", value=False, interactive=True)                
            tox.select(on_select)
            intent = gr.Checkbox(label="Intent", value=False, interactive=True)                
            intent.select(on_select)

demo.launch()

#### Troubleshooting
---

If you receive the following error

```
'Comprehend' object has no attribute 'detect_toxic_content'
```

this means that you likely do not have access to the Toxicity API, or you may be in the incorrect region.

If you receive the following error

```
An error occurred (NotAuthorizedException) when calling the ClassifyDocument operation: Your account is not authorized to use Prompt Classification feature.
```

This means that you do not have access to the intent classification API.
